In [ ]:
import requests
import json
import io
import sys
from PIL import Image
from requests.auth import HTTPBasicAuth
from IPython.display import display
import pandas as pd
import webbrowser
import time

username = 'lievuser'
password = 'changeme'
server = 'http://localhost:5011/'
auth = HTTPBasicAuth(username, password)

In [ ]:
def ask(instruction, function=None, system_msg="", max_new_tokens=18000, temperature=0.1, timeout=120, llm_name=None, try_next_on_failure = True):
    data = {
       "function": function,
       "instruction": instruction,
       "system_msg": system_msg,
       "max_new_tokens": max_new_tokens,
       "temperature": temperature,
       "timeout": timeout,
       "llm_name": llm_name,
       "try_next_on_failure": try_next_on_failure
    }
    load = requests.get(server+"response", data = json.dumps(data), auth=auth)
    if load.status_code == 200:
        print(f'\033[92m Response Headers\n Liev-Response-Model: {load.headers["Liev-Response-Model"]}\033[31m\n Liev-Response-Is-Failover: {load.headers["Liev-Response-Is-Failover"]}\n Liev-Response-Failed-Models: {load.headers["Liev-Response-Failed-Models"]}\033[0m')
        if "image" in load.headers['content-type']:
            return Image.open(io.BytesIO(load.content))
        else:
            return json.loads(load.text)
    else:
        sys.exit("Error calling the orchestrator. Erro code: " + str(load.status_code) + str(load.text))

### Listar todas as LLMs

In [ ]:
# List all available LLMs
# It can be retrieved by GET in the endpoint /v1/llm

json_llms = json.loads(requests.get(server+"v1/llms_and_types", auth=auth).text)

df = pd.DataFrame(json_llms)
selected_columns = ['type', 'name', 'model',  'priority', 'response_mime', 'is_external']
df_selected = df[selected_columns]
df_selected_sorted = df_selected.sort_values(by=['type', 'priority'])
df_selected_sorted = df_selected_sorted.reset_index(drop=True)
print (server+"v1/llms_and_types")
df_selected_sorted

### Listar LLMs por tipo

In [ ]:
# List all available LLMs By Type
# It can be retrieved by GET in the endpoint /v1/llm/<type>

json_llms = json.loads(requests.get(server+"v1/llms_and_types/code", headers={'Authorization': f"Bearer {access_token}"}).text)

df = pd.DataFrame(json_llms)
selected_columns = ['type', 'name', 'model',  'priority', 'response_mime']
df_selected = df[selected_columns]
df_selected_sorted = df_selected.sort_values(by=['type', 'priority'])
df_selected_sorted = df_selected_sorted.reset_index(drop=True)
print (server+"v1/llm/code")
df_selected_sorted

In [ ]:
# Prompt detection used by orchestrator
text = """
Classify the following prompt between the categories. Return just the category name. Prompt: Código C++ para if. Allowed categories: text,image,embeddings,code,audio Category:"""
system_msg="You are a content classifier"
print(ask(text+"\n.\n", system_msg=system_msg, function="text",llm_name="codellama"))

In [ ]:
# Misterious LLM with no type informed!
# The following request doesn't use the parameter "function". The orchestrator classifies the prompt and returns according.


print('\033[93m Asking for a story \033[0m')
print('\033[93m----------------------------------------------------\033[0m')

response = ask("Conte-me a história do Desastre de messênia")
print(response)

print('\033[93m Asking for a code \033[0m')
print('\033[93m----------------------------------------------------\033[0m')

response = ask("Gimme a sample C++ REST API")
print(response)

print('\033[93m Asking for something like an image \033[0m')
print('\033[93m----------------------------------------------------\033[0m')
response = ask("A photo of a delicious pizza of burrata and brie cheese")
display(response)

print('\033[93m Offending the LLM! \033[0m')
print('\033[93m----------------------------------------------------\033[0m')
response = ask("You're so ugly. What a shame!")
display(response)


print('\033[93m Asking for embbed a content! \033[0m')
print('\033[93m----------------------------------------------------\033[0m')
response = ask("Embbed the following sentence: 'I'm asking for an image, a text or a code?'")
display(response)




In [ ]:
#Text request , LLM autoselected by priority
text = """Título: O desastre do barco de migrantes na Messênia."""
system_msg="Você é um escritor brasileiro."
print(ask(text+"\nDado o título acima de um artigo imaginário, escreva um artigo com 200 palavras.\n", system_msg=system_msg, function='text'))


#Text request , with LLM specified by the llm_name parameter
text = """Título: O desastre do barco de migrantes na Messênia."""
system_msg="Você é um escritor brasileiro."
print(ask(text+"\nDado o título acima de um artigo imaginário, escreva um artigo com 200 palavras.\n", system_msg=system_msg, function='text', llm_name="claude"))

In [ ]:
#Image request , LLM autoselected by priority
instruction = """A dog with a weird hat."""
image = ask(instruction, function="image")
display(image)
#Image request , with LLM specified by the llm_name parameter
instruction = """A cat playing with a ball."""
image = ask(instruction, function="image", llm_name="stablediffusion")
display(image)

In [ ]:
# #Code request , LLM autoselected by priority
# prompt = """Escreva um programa em python que imprima todos os números divisíveis por 3 e 7 entre 1 e 100."""
# print(ask(prompt, function="code"))

#Code request , with LLM specified by the llm_name parameter
prompt = """Escreva um programa em Cobol que imprima todos os números divisíveis por 3 e 7 entre 1 e 100."""
print(ask(prompt, function="code", llm_name="openai"))

In [ ]:
#Asking for type and text, but the llms doesn't work. It gets failed over
prompt = """Como surgiram os aborígenes?"""
print(ask(prompt, function="text", llm_name="mixtral"))

In [ ]:
#Asking for type and text, the llm doesn't work and I don't want failover. It needs to fail if specific LLM is not available
# IT MUST FAIL
prompt = """Como surgiram os aborígenes?"""
print(ask(prompt, function="text", llm_name = "mixtral", try_next_on_failure=False))

In [ ]:
#Calling the LLM just by it's name.
#Sometimes the Same LLM can work with multiple types(functions), like text or code
#Some get better responses than others depending of type. That's why LLM failover priorities is made based on type.
#Because type is not passed and detection is not being made - since i'm telling what llm is - this mode DOESN'T SUPPORT failover
#Usually, who calls the llm directly knows what is doing
prompt = """Crie um plano de teste, com alguns casos de teste, para um jogo da velha"""
print(ask(prompt, llm_name = "vicuna"))

In [ ]:
#Multi LLM Text Request
prompt = """Which is the planet with most moons?"""
print(json.dumps(ask(prompt, function="text", llm_name="all"), indent=4))

In [ ]:
#Multi LLM Embeddings Request
prompt = """
Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoubtable source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular during the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit amet..", comes from a line in section 1.10.32.
"""
print(json.dumps(ask(prompt, function="embeddings", llm_name="all"), indent=4))